In [ ]:

import pandas as pd
import numpy as np

# Load the data from the ERE.xlsx file
file_path = 'ERE.xlsx'  # Replace with the correct path to your file
data_defensive = pd.read_excel(file_path, sheet_name='Sheet1', usecols=['playerName', 'x', 'y', 'outcome', 'typeId'], nrows=100)

# Filter the dataset for defensive actions based on `typeId == 7` or `typeId == 8`
defensive_actions = data_defensive[data_defensive['typeId'].isin([7, 8])]

# Simulate target coordinates for passes
defensive_actions['x_end'] = defensive_actions['x'] + np.random.uniform(-20, 20, len(defensive_actions))
defensive_actions['y_end'] = defensive_actions['y'] + np.random.uniform(-10, 10, len(defensive_actions))

# Calculate pass distance and angle
defensive_actions['pass_distance'] = np.sqrt(
    (defensive_actions['x_end'] - defensive_actions['x']) ** 2 +
    (defensive_actions['y_end'] - defensive_actions['y']) ** 2
)

defensive_actions['pass_angle'] = np.abs(
    np.arctan2(defensive_actions['y_end'] - defensive_actions['y'],
               defensive_actions['x_end'] - defensive_actions['x'])
)

# Calculate xPass
defensive_actions['xPass'] = 1 - (
    0.02 * defensive_actions['pass_distance'] +
    0.1 * defensive_actions['pass_angle']
)
defensive_actions['xPass'] = defensive_actions['xPass'].clip(0, 1)  # Ensure probabilities are between 0 and 1

# Calculate Pre-Action Danger using xPass
defensive_actions['pre_action_danger'] = defensive_actions['xPass']

# Simulate Post-Action Danger (reduced if defensive action is successful)
defensive_actions['post_action_danger'] = defensive_actions['pre_action_danger'] * np.where(defensive_actions['outcome'] == 1, 0.5, 1.0)

# Calculate xDef specifically for the player making the defensive action
# xDef = Danger prevented by the defensive action
defensive_actions['xDef'] = defensive_actions['pre_action_danger'] - defensive_actions['post_action_danger']

# Save the results for validation
validation_file_path = 'Validation_xDef_Calculations.xlsx'
defensive_actions[['pre_action_danger', 'post_action_danger', 'xDef']].to_excel(validation_file_path, index=False)

print(f"Validation results saved to '{validation_file_path}'")
